In [9]:
import pandas as pd
import numpy as np
import glob
import gzip
import os
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

**ACTUALIZAR WIP DESDE: https://docs.google.com/spreadsheets/d/1aLELrHoXPwm8CSY2a6DN6DRqoIkruHQO37JrQcaZFNM/edit?gid=916179586#gid=916179586**  
Guardar en `./wip_final.csv`

In [10]:
if os.path.exists('./diccionarios/') == False:
    os.makedirs('./diccionarios/')

In [11]:
cols = ['STATUS IMPLANTATION','Technical / Domains','Type of  rule','TABLE (en C360)','FORMAT (3.2)','NAMING','Regular Expresion', 'Conf name', 'version  FINISHED','ID RULE','CATALOG (TAXONOMY/STATIC) (3.5-6.1-6.10)']
wip = pd.read_csv('wip_final.csv', header=1)[cols].dropna(subset='version  FINISHED')
wip.tail(2)

,STATUS IMPLANTATION,Technical / Domains,Type of rule,TABLE (en C360),FORMAT (3.2),NAMING,Regular Expresion,Conf name,version FINISHED,ID RULE,CATALOG (TAXONOMY/STATIC) (3.5-6.1-6.10)
5622,IMPLANTABLE,DOMAIN,5.3-Comparison between different values ​​of d...,t_rzua_sust_kpi_link_oper_checkp,NaN,gf_kpi_max_rng_val_eval_number,NaN,RZUA_sust-kpi-link-oper-checkp_MRField.conf,17.4.0,28d2e215c3,NaN
5623,IMPLANTABLE,DOMAIN,3.2-Checking the format of the field (Data typ...,t_rzua_sust_kpi_link_operation,NaN,g_bbva_sus_cond_coord_ind_type,NaN,RZUA_sust-kpi-link-operation_MRField.conf,17.4.0,28d4dd2f94,NaN


In [12]:
def analisis(data, esquemas, wip):
    table = data['Physical name object'][0]
    esquema = 'https://artifactory.globaldevtools.bbva.com/ui/native/gl-datio-da-generic-dev-local/schemas/gl/'+ table[:6][2:] + '/master/' + table + '/latest/' + table + '.output.schema'

    print('=====================================================================')
    print('= TABLE:',table)
    print('== esquema:',esquema)

    print('=====================================================================')
    print('\n- PKs (4.2):')
    print('---------------------------------------------------------------------')
    for i in list(data.query('Key == True')['Physical Name field']):
        print(i)
    pks = str(list(data.query('Key == True')['Physical Name field'])).replace("\'",'"')
    print('\ncolumns:',pks)
    regla42 = wip[ ((wip['Technical / Domains'] == 'MVP TECHNICAL') | (wip['Technical / Domains'] == 'MVP DQ MODELLING'))  & (wip['TABLE (en C360)'] == table) & (wip['Type of  rule'].str[:3] == '4.2')]['ID RULE'].tolist()
    print('\n> regla 4.2 DQ: ', regla42)
    print('¡¡NO COPIAR ESTA!!')
    if regla42:
        campos42 = wip[ ((wip['Technical / Domains'] == 'MVP TECHNICAL') | (wip['Technical / Domains'] == 'MVP DQ MODELLING'))  & (wip['TABLE (en C360)'] == table) & (wip['Type of  rule'].str[:3] == '4.2')].NAMING.tolist()
        campos42 = campos42[0].split('\n')
        campos42 = [x for x in campos42 if x]
        print('columns:',campos42)
    
    print('\n- MANDATORY`s alphanumericos (3.1, 3.2):')
    print('---------------------------------------------------------------------')
    print('Descartes audit:\n', list(data[ (data['Mandatory'] == True) & ((data['Physical Name field'].str.contains('audit',regex=False)) | (data['Physical Name field']=='gf_process_id') | (data['Physical Name field']=='gf_run_id') | (data['Physical Name field']=='gf_odate_date')) & (data['Logical Format'].str.contains('ALPHANUMERIC',regex=False))]['Physical Name field']))
    print()
    campos_mandatory_datum = set()
    incr = 1
    for i in list(data[(data['Mandatory'] == True) & ~((data['Physical Name field'].str.contains('audit',regex=False)) | (data['Physical Name field']=='gf_process_id') | (data['Physical Name field']=='gf_run_id') | (data['Physical Name field']=='gf_odate_date')) & (data['Logical Format'].str.contains('ALPHANUMERIC',regex=False))][['Physical Name field','Logical Format']].itertuples(index=False, name=None)):
        campo = i[0]
        campos_mandatory_datum.add(campo)
        alphanumeric = i[1]
        regla32 = wip[ ((wip['Technical / Domains'] == 'MVP TECHNICAL') | (wip['Technical / Domains'] == 'MVP DQ MODELLING'))  & (wip['TABLE (en C360)'] == table) & (wip['NAMING'] == campo) & (wip['Type of  rule'].str[:3] == '3.2')]['ID RULE'].tolist()
        alphanumeric32 = wip[ ((wip['Technical / Domains'] == 'MVP TECHNICAL') | (wip['Technical / Domains'] == 'MVP DQ MODELLING')) & (wip['TABLE (en C360)'] == table) & (wip['NAMING'] == campo) & (wip['Type of  rule'].str[:3] == '3.2')]['FORMAT (3.2)'].tolist()
        print('\n',campo,alphanumeric,'\nregla 3.2 DQ',alphanumeric32,regla32,sep='-')
        if not regla32:
            print('-> ¡NO EXISTEN REGLAS!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! -> propuesta:')
            regla31 = "GL_MVP_" + table + "_3.1_01" + str(incr)
            regla32 = "GL_MVP_" + table + "_3.2_01" + str(incr)
            incr += 1
            print(regla31)
            print(regla32)


    if 'array' in data['Data Type'].str.lower().unique():
        print('\n¡contiene ARRAY! revisar esquema:', esquema)
    if 'struct' in data['Data Type'].str.lower().unique():
        print('\n¡contiene STRUCT! revisar esquema:', esquema)
    campos_mandatory_wip =  set(wip[((wip['Technical / Domains'] == 'MVP TECHNICAL') | (wip['Technical / Domains'] == 'MVP DQ MODELLING'))  & (wip['TABLE (en C360)'] == table) & (wip['Type of  rule'].str[:3] == '3.2')]['NAMING'].tolist())
    print('\nCampos mandatory datum:\n',campos_mandatory_datum)
    print('Campos mandatory wip:\n',campos_mandatory_wip)
    print('\n¡ELIMINAR reglas!:\n',campos_mandatory_wip - campos_mandatory_datum)
    
    if campos_mandatory_wip - campos_mandatory_datum:
        for c in campos_mandatory_wip - campos_mandatory_datum:
            regla31 = wip[ ((wip['Technical / Domains'] == 'MVP TECHNICAL') | (wip['Technical / Domains'] == 'MVP DQ MODELLING'))  & (wip['TABLE (en C360)'] == table) & (wip['NAMING'] == c) & (wip['Type of  rule'].str[:3] == '3.1')]['ID RULE'].tolist()
            regla32 = wip[ ((wip['Technical / Domains'] == 'MVP TECHNICAL') | (wip['Technical / Domains'] == 'MVP DQ MODELLING'))  & (wip['TABLE (en C360)'] == table) & (wip['NAMING'] == c) & (wip['Type of  rule'].str[:3] == '3.2')]['ID RULE'].tolist()
            print('-',regla31[0],sep='')
            print('-',regla32[0],sep='')
    print()

    print('\n- Reglas de catalogo actuales  - MVP DQ MODELLING (3.5):')
    print('---------------------------------------------------------------------')
    print('CATALOGOS:')
    catalogos_presentes = data[~data.Catalog.isnull()][['Physical Name field','Catalog']]
    campos_list = []
    for campo,catalogo in catalogos_presentes.values:
        print(campo,'"', catalogo,'"', sep='-')
        campos_list.append((str(campo),str(catalogo)))
    booleanos = data[data['Physical Name field'].str.endswith('ind_type') & data['Logical Format'].str.startswith('ALPHANUMERIC')]['Physical Name field'].to_list()
    if booleanos:
        for campo in booleanos:
            print(campo,'["Y","N"]',sep='-')
            campos_list.append((str(campo),'["Y","N"]'))
    
    print('\nREGLAS:')
    campos_3_5_wip =  wip[#(wip['Technical / Domains'] == 'MVP DQ MODELLING')  & 
                          (wip['TABLE (en C360)'] == table) & (wip['Type of  rule'].str[:3] == '3.5')][['NAMING','CATALOG (TAXONOMY/STATIC) (3.5-6.1-6.10)']]
    campos_list_wip = []
    for campo,catalogo in campos_3_5_wip.values:
        regla35 = wip[ (wip['TABLE (en C360)'] == table) & (wip['NAMING'] == campo) & (wip['Type of  rule'].str[:3] == '3.5')]['ID RULE'].tolist()
        catalog35 = wip[ (wip['TABLE (en C360)'] == table) & (wip['NAMING'] == campo) & (wip['Type of  rule'].str[:3] == '3.5')]['CATALOG (TAXONOMY/STATIC) (3.5-6.1-6.10)'].tolist()
        print(campo,catalogo,'\nregla 3.5 DQ',catalog35,regla35,sep='-')
        campos_list_wip.append((str(campo),str(catalog35), regla35))
    esquemas.append(esquema)

    print('\n- Reglas de catalogo - CAMBIOS A REALIZAR:')
    print('---------------------------------------------------------------------')
    campos_3_5_wip = pd.DataFrame(campos_list_wip,columns=['campo','cat','r'])
    campos_3_5 = pd.DataFrame(campos_list,columns=['campo','cat'])
    campos_3_5_vs_WIP = campos_3_5.merge(campos_3_5_wip,on='campo',how='left',suffixes=('', '_WIP'))
    campos_3_5_vs_WIP.cat= campos_3_5_vs_WIP.cat.str.split('-').apply(lambda x:x[0])
    print('-NUEVAS reglas 3.5:')
    print(campos_3_5_vs_WIP[campos_3_5_vs_WIP.cat_WIP.isnull()][['campo','cat']].values)
    campos_3_5_vs_WIP_ACTUALES = campos_3_5_vs_WIP[~campos_3_5_vs_WIP.cat_WIP.isnull()]
    campos_3_5_vs_WIP_ACTUALES.cat_WIP = campos_3_5_vs_WIP_ACTUALES.cat_WIP.str.strip().str.split("'").apply(lambda x:x[1])
    campos_3_5_vs_WIP_ACTUALES.cat.str.strip().replace('["Y","N"]','Y/N',inplace=True)
    campos_3_5_vs_WIP_ACTUALES.r = campos_3_5_vs_WIP_ACTUALES.r.apply(lambda x:x[0])
    campos_3_5_vs_WIP_ACTUALES['coinciden'] = campos_3_5_vs_WIP_ACTUALES.cat_WIP == campos_3_5_vs_WIP_ACTUALES.cat

    print('\n-MODIFICAR reglas 3.5:')
    print(campos_3_5_vs_WIP_ACTUALES[campos_3_5_vs_WIP_ACTUALES['coinciden']==False][['campo','cat','r']].values)


In [13]:
dict_data = {}
esquemas = []
colnames = [
    "index",'Field ID','Physical Name field','Description field','Logical name field','Data Type','Key','Catalog','Format','Logical Format','Mandatory',
    'Default Value','Tags','Field Position in the object','Generated field','Tokenization type','Physical name of source object','Source field','Data type of source field',
    'Format of source field','Registration Date','Object ID','Concept ID','Security class','Security label','Tokenized at datasource','Tokenized','Locale','Technical comments',
    'Source data model code','Source field length','Trusted datasource','Version','Physical name object','Model code','Deployment type','Country','Data System ID','Name Data System',
    'System code','Storage Type','Storage Zone','Relevante','Optional','Deleted','Farm','Verticalized','Deprecated','Partition Column Ind','Field Source desc','Need for catalogue','Catalog source','Catalog ID'
    ]
# for ruta in glob.glob('./diccionarios/*.csv'):
for ruta in glob.glob('./diccionarios/*.csv.gz'):
    print('\n', ruta)
    try:
        data = pd.read_csv(ruta, compression='gzip', sep=';').reset_index()
        # data.columns = colnames
        if data.index[0]>1:
            data = pd.read_csv(ruta, compression='gzip', sep=';', header=None, skiprows=1).reset_index()
            data.columns = colnames
    except:
        data = pd.read_csv(ruta, compression='gzip', sep=';', header=None, skiprows=1).reset_index()
        data.columns = colnames
    
    dict_data[data['Physical name object'][0]] = data
    
    analisis(data, esquemas, wip)


 ./diccionarios\t_kcpg_budget_cust_trip.csv.gz
= TABLE: t_kcpg_budget_cust_trip
== esquema: https://artifactory.globaldevtools.bbva.com/ui/native/gl-datio-da-generic-dev-local/schemas/gl/kcpg/master/t_kcpg_budget_cust_trip/latest/t_kcpg_budget_cust_trip.output.schema

- PKs (4.2):
---------------------------------------------------------------------
gf_cutoff_date
g_entity_id
g_country_id
g_trip_register_id

columns: ["gf_cutoff_date", "g_entity_id", "g_country_id", "g_trip_register_id"]

> regla 4.2 DQ:  ['GL_STR_t_kcpg_budget_cust_trip_4.2_000']
¡¡NO COPIAR ESTA!!
columns: ['g_entity_id', 'g_country_id', 'gf_cutoff_date', 'g_trip_register_id']

- MANDATORY`s alphanumericos (3.1, 3.2):
---------------------------------------------------------------------
Descartes audit:
 []


-g_entity_id-ALPHANUMERIC(6)-
regla 3.2 DQ-['ALPHANUMERIC(6)']-['GL_MVP_t_kcpg_budget_cust_trip_3.2_000']

-g_country_id-ALPHANUMERIC(2)-
regla 3.2 DQ-['ALPHANUMERIC(2)']-['GL_MVP_t_kcpg_budget_cust_trip_3.2_00

In [14]:
dict_data.keys()

dict_keys(['t_kcpg_budget_cust_trip', 't_re4m_physical_interaction'])

In [15]:
for i in esquemas: print(i)

https://artifactory.globaldevtools.bbva.com/ui/native/gl-datio-da-generic-dev-local/schemas/gl/kcpg/master/t_kcpg_budget_cust_trip/latest/t_kcpg_budget_cust_trip.output.schema
https://artifactory.globaldevtools.bbva.com/ui/native/gl-datio-da-generic-dev-local/schemas/gl/re4m/master/t_re4m_physical_interaction/latest/t_re4m_physical_interaction.output.schema


In [16]:
# dict_data['t_l9ct_co_asset_rel_activity']